# Imports

we want to import openCV, mediapipe, numpy

In [1]:
import cv2
import mediapipe as mp
import numpy

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
  
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 


In [6]:
cap = cv2.VideoCapture('./videos/Cross.MOV')
frameCount = 0
jumpFrame = -1
pendingSave = False
clipStartFrame = 0

clipNum = int(input("Enter current recording num"))
# Set mediapipe model 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        if jumpFrame >= 0:
            jumpFrame -= 1

        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        frame = []
        if jumpFrame >= 0:
            if results.pose_landmarks:
                for res in results.pose_landmarks.landmark:
                    frame.append(res.x)
                    frame.append(res.y)
                    frame.append(res.z)
                    frame.append(res.visibility)
            else:
                for i in range(132):
                    frame.append(0)

            # frames.append(frame)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen

        if pendingSave:
            cv2.putText(image, text= str("Frames: " + str(clipStartFrame) + " - " + str(frameCount) + ". Press c to save these frames to " + str(clipNum) + ".npy"), org=(700, 800),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,255),
                thickness=2, lineType=cv2.LINE_AA)
            
            cv2.putText(image, text= str("Press x to discard these frames"), org=(700, 850),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,255),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.putText(image, text= str("Action #" + str(clipNum) + ". Saving to " + str(clipNum) + ".npy"), org=(700, 1000),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,0,0),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.putText(image, text= str(frameCount), org=(500, 300),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,0),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        
        wait = 10 if jumpFrame >= 0 else 0
        
        key = cv2.waitKey(wait)
        if key == ord('q'):
            break

        if pendingSave and key == ord('c'):
            pendingSave = False
            clipNum += 1
            print("Saved")
            # print(frames)
            print("---------------")
            # frames = []
            continue
        if pendingSave and key == ord('x'):
            pendingSave = False
            print("Discarded")
            print("---------------")
            # frames = []
            frameCount = clipStartFrame
            cap.set(cv2.CAP_PROP_POS_FRAMES, clipStartFrame)
            continue
        if not pendingSave and jumpFrame < 0:
            if key == ord('p'):
                frameCount -= 1
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
                continue
            if key == ord('d'):
                frameCount += 100
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
                continue
            if key == ord('a'):
                frameCount -= 100
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
                continue
            if key == ord('s'):
                clipStartFrame = frameCount
                jumpFrame = 25
                pendingSave = True
                continue
            else:
                frameCount += 1
                continue

            # Quit when 'q' is pressed

        # print("pog")
        frameCount += 1

    cap.release()
    cv2.destroyAllWindows()


# print(frames)

Saved
---------------
